# III. HÁZI FELADAT

Ezt a feladatsort Python nyelven kell megoldani. A kimeneti adatok ábrázolásához minden esetben használjuk a jupyter notebookot! Az ábrák legyenek áttekinthetőek, és szerepeljen rajtuk tengelyfelirat!

Az feladatok megoldásához szükség lesz az `ahkab` nevű áramkörszimulációs csomagra, és a `SchemDraw` nevű, kapcsolási rajzok készítésére szolgáló csomagra, melyek legfrissebb verziói `pip` segítségével installálhatók:

```
$ pip install git+https://github.com/ahkab/ahkab.git
$ pip install SchemDraw
```

# Passzív áramkörök numerikus vizsgálata

Lineáris, időben állandó fizikai rendszerek vizsgálatakor gyakran foglalkozunk ideális elektronikus áramkörökkel, hiszen az elméleti érdekességeken túl ezek a gyakorlatban is igen fontosak. Az alábbi feladatok passzív elektronikai alkatrészekből (ellenállás, kondenzátor és induktivitás) megépíthető áramkörök viselkedését vizsgálják, de az alkalmazott módszerek a jelfeldolgozás területén általánosan használhatók. A feladatok megoldása során az `ahkab` áramkörszimulációs csomagot használjuk, mely nagy időfelbontással számítja ki az áramkör egyes csomópontjaiban megjelenő jelalakokat (időfüggő feszültségeket). Az ilyen szimuláció közelebb áll a valódi analóg áramkörök működéséhez. A későbbiekben olyan, ún. diszkrét idejű rendszerekkel is fogalkozunk majd, melyek a digitális jelfeldolgozás modelljének tekinthetők.

A passzív elektronikai altrészek viselkedése a következő egyszerű fizikai törvényekkel írhatók le, ahol $u = u(t)$ az időfüggő feszültség, $i = i(t)$ az áram, $R$ az elleneállás, $C$ a kapacitás és $L$ az induktivitás:

* Ellenállás: $$ u = R i $$

* Kondenzátor: $$ \frac{du}{dt} = \frac{1}{C} i $$

* Induktivitás: $$ u = L \frac{di}{dt} $$

Konkrét áramkörök esetében az egyes komponensekre felírt differenciálegyenleteket a [Kirchhoff-törvények](https://hu.wikipedia.org/wiki/Kirchhoff-törvények) kapcsolják rendszerbe. Az egyenletek megoldásakor valamelyik bemeneti jelalakot (feszültség- vagy áramérték időbeli változását) előírjuk, és a rendszer erre adott válaszát, azaz egy kimeneti jelalak időfejlődését keressük.

Érdekes elméleti kérdés, hogy egy rendszer milyen választ ad egy nagyon rövid ideig tartó vizsgálóimpulzus esetében. Matematikailag az ilyen vizsgálóimpulzusokat Dirac-delta függvénnyel írjuk le, és ismert, hogy egy lineáris, időinvariáns rendszer válaszfüggvénye tetszőleges gerjesztés esetében meghatározható az **impulzusválasz** (más néven Green-függvény) és a gerjesztés konvolúciójaként:

$$
u_\mathrm{out}(t) = \int_{-\infty}^{t} u_\mathrm{in}(\tau) g(t-\tau) d\tau,
$$

ahol az, hogy az integrálás $t$-ig tart a rendszer kauzalitását fejezi ki.

Lineáris időinvariáns rendszereket érdemes Fourier-kifejtésben vizsgálni. Ilyekor a rendszer szinuszos bemenő gerjesztésekre adott válaszát keressük, ami a tranziens viselkedés lecsengése után a linearitás miatt azonos frekvenciájú, szintén szinuszos jelalak lesz. Egy rendszer **átviteli függvénye** megadja, hogy egy $f$ frekvenciájú szinuszos gerjesztés esetében a rendszer mekkora csillagpítást, illetve fázistolást okoz. Ismert, hogy Fourier-térben a konvolúció művelete a gerjesztés és az impulzusválasz Fourier-transzformáltjának szorzataként írható fel, és ebből az is következik, hogy az átviteli függvény az impulzusválasz Fourier-transzformáltja.

A feladatok során passzív áramköri elemekből összállított analóg szűrőket vizsgálunk, melyek az ún. Butterworth-topológiát követik. Az interneten elérhetők olyan eszközök, melyekkel különböző átviteli karakterisztikájú passzív vagy aktív szűrők tervezhetők, ld. pl.:    
    
* https://rf-tools.com/lc-filter/
* https://www.analog.com/designtools/en/filterwizard/

Az alábbi cellák olyan függvényeket definiálnak, melyek segítségével az ahkab programcsomag részletes ismerete nélkül is kiszámíthajuk néhány áramkör bemenő jelre adott válaszfüggvényét.

In [ ]:
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

import ahkab
from ahkab import new_ac, run
from ahkab.circuit import Circuit
from ahkab import circuit, printing, time_functions

import SchemDraw as schem
import SchemDraw.elements as elem

In [ ]:
def run_transient(circuit, fun, tstart=0, tstop=1e-2, tstep=1e-5):
    gnd = circuit.get_ground_node()
    circuit.add_vsource("V_in", n1="n_in", n2=gnd, dc_value=5, ac_value=1, function=fun)
    t = ahkab.new_tran(tstart=tstart, tstop=tstop, tstep=tstep, x0=None)
    r = ahkab.run(circuit, an_list=[t])
   
    time = r['tran']['T']
    source = fun(r['tran']['T'])
    response = r['tran']['Vn_out']
   
    circuit.remove_elem("V_in")
   
    return time, source, response

## Egyszerű aluláteresztő RC-kör

In [ ]:
def rc_lowpass():
    c = circuit.Circuit(title="RC-tag")
    
    c.add_resistor("R1", n1="n_in", n2="n_out", value=600)
    c.add_capacitor("C1", n1="n_out", n2=c.gnd, value=210e-9)
    
    return c

def draw_rc_lowpass():
    d = schem.Drawing()
    V_in = d.add(elem.SOURCE_SIN, d='up')
    R = d.add(elem.RES, d='right', label='600')
    C = d.add(elem.CAP, d='down', botlabel='210n')
    d.add(elem.LINE, to=V_in.start)
    d.add(elem.LINE, d='right', xy=R.end)
    d.add(elem.DOT, label='a')
    d.add(elem.LINE, d='right', xy=C.end)
    d.add(elem.DOT, label='b')
    d.draw()
    
draw_rc_lowpass()

## 4-ed rendű Butterworth-féle aluláteresztő szűrő

In [ ]:
def butterworth_lowpass():
    c = circuit.Circuit(title="4-ed rendű Butterworth-féle aluláteresztő szűrő")
    
    c.add_resistor("R1", n1="n_in", n2="n2", value=600)
    c.add_inductor("L1", n1="n2", n2="n3", value=15e-3)
    c.add_capacitor("C1", n1="n3", n2=c.gnd, value=200e-9)
    c.add_inductor("L2", n1="n3", n2="n_out", value=61e-3)
    c.add_capacitor("C2", n1="n_out", n2=c.gnd, value=150e-9)
    c.add_resistor("R2", n1="n_out", n2=c.gnd, value=1.1e3)
    
    return c

def draw_butterworth_lowpass():
    d = schem.Drawing()
    
    V_in = d.add(elem.SOURCE_SIN, d='up')
    R1 = d.add(elem.RES, d='right', label='$R_1$\n600')
    L1 = d.add(elem.INDUCTOR2, d='right', label='$L_1$\n15m')
    C1 = d.add(elem.CAP, d='down', label='$C_1$\n200n')
    L2 = d.add(elem.INDUCTOR2, d='right', xy=C1.start, label='$L_2$\n61m')
    C2 = d.add(elem.CAP, d='down', label='$C_2$\n150n')
    d.add(elem.LINE, d='right', xy=C2.start)
    R2 = d.add(elem.RES, d='down', label='$R_2$\n1.1k')
    for i in range(4): d.add(elem.LINE, d='left')
    d.add(elem.LINE, d='right', xy=R2.start)
    d.add(elem.DOT, label='$V_\mathrm{out}$')
    d.add(elem.LINE, d='right', xy=R2.end)
    d.add(elem.DOT)
    
    d.draw()
    
draw_butterworth_lowpass()

In [ ]:
def butterworth_bandpass():
    c = Circuit('Butterworth-féle 1kHz-es sáváteresztő szűrő')
    
    c.add_resistor('R1', 'n_in', 'n2', 50.)
    c.add_inductor('L1', 'n2', 'n3', 12e-3)
    c.add_capacitor('C1', 'n3', 'n4', 2.2e-6)
    c.add_inductor('L2', 'n4', c.gnd, 2e-3)
    c.add_capacitor('C2', 'n4', c.gnd, 13e-6)
    c.add_inductor('L3', 'n4', 'n5', 39e-3)
    c.add_capacitor('C3', 'n5', 'n6', 680e-9)
    c.add_inductor('L4', 'n6', c.gnd, 2e-3)
    c.add_capacitor('C4', 'n6', c.gnd, 13e-6)
    c.add_capacitor('C5', 'n7', 'n_out', 2.2e-6)
    c.add_inductor('L5', 'n6', 'n7', 12e-3)
    c.add_resistor('R2', 'n_out', c.gnd, 50.)
    
    return c

def draw_butterworth_bandpass():
    d = schem.Drawing()
    
    V_in = d.add(elem.SOURCE_SIN, d='up')
    R1 = d.add(elem.RES, d='right', label='$R_1$\n50')
    L1 = d.add(elem.INDUCTOR2, d='right', label='$L_1$\n12m')
    C1 = d.add(elem.CAP, d='right', label='$C_1$\n2.2$\mu$')
    L2 = d.add(elem.INDUCTOR2, d='down', label='$L_2$\n2m')
    d.add(elem.LINE, d='right', xy=L2.start)
    C2 = d.add(elem.CAP, d='down', label='$C_2$\n13$\mu$')
    L3 = d.add(elem.INDUCTOR2, d='right', xy=C2.start, label='$L_3$\n39m')
    C3 = d.add(elem.CAP, d='right', label='$C_3$\n680n')
    L4 = d.add(elem.INDUCTOR2, d='down', label='$L_4$\n2m')
    d.add(elem.LINE, d='right', xy=L4.start)
    C4 = d.add(elem.CAP, d='down', label='$C_4$\n13$\mu$')
    L5 = d.add(elem.INDUCTOR2, d='right', xy=C4.start, label='$L_5$\n12m')
    C5 = d.add(elem.CAP, d='right', label='$C_5$\n2.2$\mu$')
    R2 = d.add(elem.RES, d='down', label='$R_2$\n50')
    for i in range(9): d.add(elem.LINE, d='left')
    d.add(elem.LINE, d='right', xy=R2.start)
    d.add(elem.DOT, label='$V_\mathrm{out}$')
    d.add(elem.LINE, d='right', xy=R2.end)
    d.add(elem.DOT)
    
    d.draw()
    
draw_butterworth_bandpass()

# Feladatok

## 1. feladat - Mérőjelek szimulációja

Analóg áramkörök kísérleti, szimulált és analitikus vizsgálatakor gyakran alkalmazunk ún. mérőjeleket. Írjunk függvényeket, melyek megvalósítják

* a tetszőleges frekvenciájú, amplitúdójú és fázisú szinuszos mérőjelet.
* a tetszőleges periódusidejű és amplitúdójú négyszögjelet.
* az egységimpulzust.
* az egységugrást.

Ábrázoljuk a mérőjeleket grafikonon. Interpoláció segítségével mintavételezzük a megvalósított mérőjeleket állandó időbeli felbontással. Milyen mintavételezési frekvencia szükséges egy $f$ frekvenciájú szinuszhullám mintavételezéséhez?

In [ ]:
%pylab inline
from scipy import signal
import numpy as np
from sympy import DiracDelta
from scipy.interpolate import interp1d

In [ ]:
# mérőjeleket megvalósítő függvények

#Sinus-os függvények

def szin(A,freq,gam,t): # A-val beállítom az amplitúdót, freq-el adom meg a frekvenciát
    omega = 2*np.pi*freq           
    sig = A* np.sin(omega*t+gam)
    
    return sig, t

def Nszög(A,f,t): # x mintavételezés tartománya
    # Plot the square wave signal
    sig = A * signal.square(2 * np.pi * f * t)
    
    return sig,t


def Eimp(n,k,t):
    Eimp1 = signal.unit_impulse(n,k) # n-db, k-helyen, k nak 'mid'-et adva pont középre rakja
    return Eimp1,t

def US(x1,x2):
    t = np.zeros(())
    US1 = np.empty(())
    US1 = np.heaviside(x1,x2)    #x1-et (array) hasonlítja össze 0 val, x2- az az érték amit akkor ad ha x1 = 0 val
    return US1,t
  

modSzin = szin(1,100,0,np.linspace(0,10e-3,500))[0]#Amplitudo,Frekvencia,Fázis, Tartomány

modNszög_t = Nszög(1,5,linspace(-1, 1, 1000))[1]   #utolsó argumentum adja frekvenciát,Amplitudó
modNszög_sig = Nszög(1,5,linspace(-1, 1, 1000))[0]

Eimp_sign = Eimp(30,1,np.linspace(0,10e-3,1000))[0]    #Lépéshossz,Egységimp helye

x1=[1,3,6,0,1,3,0,2,5,-3,-6-5]
xt = linspace(0,len(x1),len(x1))
US_sign = US(x1,0.5)[0]

In [ ]:
plt.figure(num=None, figsize=(16, 12), dpi=80, facecolor='w', edgecolor='r')  
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.2)

plt.subplot(2, 2, 1)
plt.title("Sinus")
plot(modSzin) # Sinus 
plt.grid(True)

plt.subplot(2, 2, 2)
plt.title("Squer sign")
plot(modNszög_t,modNszög_sig) # Négyszögjel
plt.grid(True)

plt.subplot(2, 2, 3)
plt.title("Unit impulse")
plot(Eimp_sign) #Egységimpulzus
plt.grid(True)

plt.subplot(2, 2, 4)
plt.title("Heaviside")
plot(xt,US_sign)
plt.grid(True)

## 2. feladat - Mérőjelekre adott válaszok analízise

Szimuláljuk a mérőjelek hatását a fenti függvényekben definiált áramkörökre:

* Butterworth-féle aluláteresztő szűrő
* Butterworth-féle 1kHz-es sáváteresztő szűrő

Egységurás és impulzus esetén ügyeljünk arra, hogy a szimulált áramkörök viselkedését kiértékelő algoritmus csak folytonos, szakadással nem rendelkező függvények esetében működik, így célszerű a mérőjeleket az `interp1d` osztály segítségével "folytonossá" alakítani.

A paraméterezendő függvényeket legegyszerűbben ún. lambda kifejezésként adhatjuk át a tranziens viselkedést kiszámító `run_transient` függvénynek.

Az alábbi példa az RC-tag válaszfüggvényét mutatja 1kHz-es szinuszos gerjesztés esetén.

In [ ]:
#Példa feladat
c = rc_lowpass()
t, s, r = run_transient(c, lambda t: np.sin(1000 * 2 * np.pi * t), tstart=0, tstop=10e-3)

plt.plot(t * 1e3, s)
plt.plot(t * 1e3, r)
plt.xlabel("ms")
plt.ylabel("V")

## Ábrázolások saját szinusszal

In [ ]:
c = rc_lowpass()
t, s, r = run_transient(c, lambda t: szin(1,1000,0,t)[0], tstart=0, tstop=10e-3)
plt.plot(t * 1e3, s)
plt.plot(t * 1e3, r)
plt.xlabel("ms")
plt.ylabel("V")

In [ ]:
c = butterworth_bandpass()
t, s, r = run_transient(c, lambda t: szin(1,1000,0,t)[0], tstart=0, tstop=10e-3)
plt.plot(t * 1e3, s)
plt.plot(t * 1e3, r)
plt.xlabel("ms")
plt.ylabel("V")

In [ ]:
#Saját szinusszal
c = butterworth_lowpass()
t, s, r = run_transient(c, lambda t: szin(1,1000,0,t)[0], tstart=0, tstop=10e-3)
plt.plot(t * 1e3, s)
plt.plot(t * 1e3, r)
plt.xlabel("ms")
plt.ylabel("V")

## Ábrázolások saját Négyszögjellel

In [ ]:
c = rc_lowpass()
t, s, r = run_transient(c, lambda t:Nszög(1,101,t)[0], tstart=0, tstop=10e-2)
plt.plot(t * 1e3, s)
plt.plot(t * 1e3, r)
plt.xlabel("ms")
plt.ylabel("V")

In [ ]:
c = butterworth_bandpass()
t, s, r = run_transient(c, lambda t: Nszög(1,102,t)[0], tstart=0, tstop=10e-3)
plt.plot(t * 1e3, s)
plt.plot(t * 1e3, r)
plt.xlabel("ms")
plt.ylabel("V")

In [ ]:
c = butterworth_lowpass()
t, s, r = run_transient(c, lambda t: Nszög(1,104,t)[0], tstart=0, tstop=10e-3)
plt.plot(t * 1e3, s)
plt.plot(t * 1e3, r)
plt.xlabel("ms")
plt.ylabel("V")

In [ ]:
Nszög(1,5,linspace(-1, 1, 1000))[1]

c = butterworth_lowpass()
t, s, r = run_transient(c, Nszög(1,5,linspace(-1, 1, 1000))[0], tstart=0, tstop=10e-3)
plt.plot(t * 1e3, s)
plt.plot(t * 1e3, r)
plt.xlabel("ms")
plt.ylabel("V")

## Ábrák saját egységimpulzussal

In [ ]:
#Újra definiálom, hogy könnyebben kezelhessem az interpolációt
def kd(t, t0 = 0):
    return 1 * (t == t0)

t_kd = linspace(-1, 1, 10001)

kdi = interp1d(t_kd, kd(t_kd), kind = 'linear')

In [ ]:
c_0 = rc_lowpass()
c_1 = butterworth_bandpass()
c_2 = butterworth_lowpass()

plt.figure(num=None, figsize=(16, 12), dpi=80, facecolor='w', edgecolor='r')  
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.2)

plt.subplot(2, 2, 1)
plt.title("Unit impulse")
plot(t_kd, kdi(t_kd))


plt.subplot(2, 2, 2)
plt.title("Lowpass")
t, s, r = run_transient(c_0, kdi, tstart=0, tstop=10e-3)
plt.plot(t * 1e3, s)
plt.plot(t * 1e3, r)
plt.xlabel("ms")
plt.ylabel("V")
xlim(0,4)

plt.subplot(2, 2, 3)
plt.title("Butterworth bandpass")
t, s, r = run_transient(c_1, kdi, tstart=0, tstop=10e-3)
plt.plot(t * 1e3, s)
plt.plot(t * 1e3, r)
plt.xlabel("ms")
plt.ylabel("V")

plt.subplot(2, 2, 4)
plt.title("Butterworth lowpass")
t, s, r = run_transient(c_2, kdi, tstart=0, tstop=10e-3)
plt.plot(t * 1e3, s)
plt.plot(t * 1e3, r)
plt.xlabel("ms")
plt.ylabel("V")
xlim(0,2.5)

### Ábrák saját lépcsőfüggvénnyel

In [ ]:
# Újra definiálom, hogy könnyebben kezelhessem az interpolációt
def hs(t, t0 = 0):
    return 1 * (t >= t0)

t_hs = linspace(-1, 1, 10001)

hsi = interp1d(t_hs, hs(t_hs), kind = 'linear')

In [ ]:
plot(t_hs, hsi(t_hs))

In [ ]:
c = rc_lowpass()
t, s, r = run_transient(c, hsi, tstart=0, tstop=10e-3)
plt.plot(t * 1e3, s)
plt.plot(t * 1e3, r)
plt.xlabel("ms")
plt.ylabel("V")


In [ ]:
c = butterworth_bandpass()
t, s, r = run_transient(c, hsi, tstart=0, tstop=10e-3)
plt.plot(t * 1e3, s)
plt.plot(t * 1e3, r)
plt.xlabel("ms")
plt.ylabel("V")

In [ ]:
c = butterworth_lowpass()
t, s, r = run_transient(c, hsi, tstart=0, tstop=10e-3)
plt.plot(t * 1e3, s)
plt.plot(t * 1e3, r)
plt.xlabel("ms")
plt.ylabel("V")

## 3. feladat - Átviteli függvény meghatározása

Szimuláljuk a rendszer kimenetét különböző fkrevenciájú szinuszos bemenőjelek esetén. Határozzuk meg a kimenet amplitúdóját és fáziseltolódását a frekvencia függvényében. Ügyeljünk arra, hogy a kimeneti jel amplitúdóját a tranziens szakasz eltűnése után kell megmérni. Ábrázoljuk az átviteli függvényt.

In [ ]:
# Átviteli függvény meghatározása

In [ ]:
# Átviteli függvény ábrázolása

## 4. feladat - Az impulzusválasz tulajdonságainak vizsgálata

Szimuláljuk a fent használt sáváteresztő szűrő impulzusra adott válaszát. Mintavételezzük a válaszfüggvényt egyenletes időlépéssel, majd határozzuk meg a Fourier-transzformáltát. Hogyan viszonyul az impulzusválasz Fourier-transzformáltja az előző feladatban meghatározott átviteli függvényhez?

In [ ]:
# Sáváteresztő szűrő impulzusválasza

Interpoláljuk az impulzusválaszt egyenletes időfelbontású rácsra, majd határozzuk meg a Fourier-transzformáltját. Hasonlítsuk ezt össze a 3. feladatban meghatározott átviteli függvénnyel.

In [ ]:
# Sáváteresztő szűrő impulzusválaszának Fourier-transzformáltja

Határozzuk meg az *aluláteresztő szűrő* impulzusválaszát, majd interpoláljuk időben egyenletes rácsra. Számoljuk ki a korábban már használt négyszögjel vagy más jelalak értékeit ugyanezen rácson. A `numpy.convolve` függvény segítségével számítsuk ki az impulzusválasz és a jelalak konvolúcióját. Hasonlítsuk ezt össze a 2. feladatban használt jelalakra kapott válasszal.

In [ ]:
# Aluláteresztő szűrő impulzusválasza

In [ ]:
# Az impulzusválasz és a vizsgálójelek konvolúciójának meghatározása

## 5. feladat - Zajok vizsgálata

Vizsgáljuk meg a sáváteresztő szűrő zaj jellegű gerjesztésre adott válaszát. Ehhez állítsunk elő ún. fehérzajt. A fehérzajt az jellemzi, hogy minden Fourier-komponense azonos amplitúdójú, de a komponensekhez tartozó fázisok véletlen eloszlásúak.

Ábrázoljuk a fehérzajt kinagyítva is, majd hasonlítsuk össze más, zaj jellegű függvényekke:

* uniform zaj: az időfüggő jel értéke egyenletes eloszlásból származik
* Poisson-zaj: az időfüggő jel értéke a Poisson-eloszlást követi

Határozzuk meg a sáváteresztő és az aluláteresztő szűrők válaszfüggvényét a fehérzajra. Vizsgáljuk meg a fehérzajra adott válasz Fourier-transzformáltját, és hasonlítsuk össze a hálózat átviteli függvényével.

In [ ]:
# Zaj egyenletes eloszlásból generálva

In [ ]:
# Egyenletes eloszlásból származó zaj Fourier-transzformáltja

In [ ]:
# Poisson-zaj előállítása

In [ ]:
# Poisson-zaj Frourier-transzformáltja

In [ ]:
# Fehérzaj előállítása

In [ ]:
# Sáváteresztő szűrő fehérzajra adott válaszának meghatározása

In [ ]:
# A fehérzajra adott válasz Fourier-transzformáltja